In [1]:
import sys
import re
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

In [3]:
# https://www.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&region=184723
# https://www.cian.ru/cat.php?deal_type=sale&offer_type=flat&region=184723

In [4]:
base_url='https://www.cian.ru/cat.php'
req_param='deal_type=sale&offer_type=flat&region=184723'

In [5]:
from lib.parser import AdsListParser

In [6]:
class CianParser(AdsListParser):
    
    def __init__(self,driver,base_url='https://www.cian.ru/cat.php',):
        super().__init__(
            driver=driver, 
            base_url=base_url,
            item_tag=['article',{'data-name':'CardComponent',},],
            paginator_url_param='p',
        )
        logging.info('CianParser: downloader init')
    
    def _is_last_page(self,root,p): 
        try:
            paginator = root.find('div',{'data-name':'Pagination',})
            pl = [ tag.text for tag in root.find_all('li') ][-1]
            return str(p)==pl
        except Exception as e:
            logging.warning(f'CianParser: parse pagination error: {e}')
            return True
        
    def _parse_item(self,tag): 
        return { 
             'OfferTitle': self._get_title(tag),
          'OfferSubtitle': self._get_subtitle(tag),
               'Deadline': self._get_deadline(tag),
              'MainPrice': self._get_price(tag),
              'PriceInfo': self._get_price_m(tag),
               'GeoLabel': self._get_adr(tag),
              'TimeLabel': self._get_ts( tag ),  
               'LinkArea': self._get_link( tag ),
            'Description': self._get_descr(tag),
                        
            }
    
    @staticmethod
    def _get_deadline(tag):
        try:
            return tag.find('span',{'data-mark':'Deadline',}).text
        except:
            return None
        
    @staticmethod
    def _get_title(tag):
        try:
            return tag.find('span',{'data-mark':'OfferTitle',}).text
        except:
            return None
        
    @staticmethod
    def _get_subtitle(tag):
        try:
            return tag.find('span',{'data-mark':'OfferSubtitle',}).text
        except:
            return None
        
    @staticmethod
    def _get_price(tag):
        try:
            return tag.find('span',{'data-mark':'MainPrice',}).text
        except:
            return None

    @staticmethod
    def _get_price_m(tag):
        try:
            return tag.find('p',{'data-mark':'PriceInfo',}).text
        except:
            return None
        
    @staticmethod
    def _get_adr(tag):
        try:
            return [ t.text for t in tag.find_all('a',{'data-name':'GeoLabel',}) ]
        except:
            return []
    
    @staticmethod
    def _get_ts(tag):
        try:
            tag_ = tag.find('div',{'data-name':'TimeLabel',})
            return [ t.text for t in tag_.find_all('span') ]
        except:
            return []
        
    @staticmethod
    def _get_link(tag):
        try:
            return tag.find('div',{'data-name':'LinkArea',}).find('a').attrs['href']
        except:
            return None
        
    @staticmethod
    def _get_descr(tag):
        try:
            return tag.find('div',{'data-name':'Description',}).text
        except:
            return None
    

---

In [7]:
base_url = 'https://sevastopol.cian.ru/kupit-kvartiru/'

In [8]:
profile_path = '/home/mechanoid/.mozilla/firefox/p144xo2m.default-release'
from lib.downloader import DownloaderSeleniumFirefox

# from lib.downloader import DownloaderSimple

df,html = CianParser(
        base_url = base_url,
        driver=DownloaderSeleniumFirefox(profile_path),
        # driver=DownloaderSimple(),
    ).load(req_param='', keep_html=True, page_limit=3 )

print(len(df))

[INFO    ] 2022-09-07 14:11:27 | DownloaderSeleniumFirefox: downloader init
[INFO    ] 2022-09-07 14:11:27 | DownloaderSeleniumFirefox: open virtual browser
[INFO    ] 2022-09-07 14:11:29 | AdsListParser: downloader init
[INFO    ] 2022-09-07 14:11:29 | CianParser: downloader init
[INFO    ] 2022-09-07 14:11:29 | AdsListParser: start read and parse pages...
[INFO    ] 2022-09-07 14:11:34 | AdsListParser: read page 1
[INFO    ] 2022-09-07 14:11:37 | AdsListParser: read page 2
[INFO    ] 2022-09-07 14:11:38 | DownloaderSeleniumFirefox: close virtual browser
50


In [ ]:
with open('tmp/cian.html','wt') as f: f.write(html[0])

In [ ]:
# html

In [9]:
df

,OfferTitle,OfferSubtitle,Deadline,MainPrice,PriceInfo,GeoLabel,TimeLabel,LinkArea,Description,page,ts
0,"2-комн. кв., 61,38 м², 4/10 этаж",Сдача корпуса 4 кв. 2023,сдача ГК: 4 кв. 2023 года,4 910 400 ₽,80 000 ₽/м²,"[Севастополь, р-н Гагаринский, Казачья Бухта мкр, Скифия ЖК]","[4 часа назад, сегодня, 10:04]",https://sevastopol.cian.ru/sale/flat/273181124/,"Прекрасная квартира с городской пропиской в новом доме у моря. \n5 минут до пляжа! Остановка в шаговой доступности , 15 минут до центра города!\n\nДвухконтурный газовый котел. Индивидуальное отопление, которое позволит Вам не только сократить коммунальные платежи, но и создать комфортную температуру в квартире. \nУстановлены приборы учета (газ, вода, электричество)\nВысокие потолки (2.8 м) и увеличенные стеклопакеты визуально делают квартиру еще более светлой и просторной.\n\nТерритория жилого комплекса полностью огорожена. \nУютный зеленый двор с удобными скамейками для отдыха и детская площадка с каруселями и горками, которая обязательно понравится Вашим детям!\n\nМоре и пляж, магазины и социальные объекты (новая школа и детский сад), исторические и культурные объекты Севастополя все это рядом с домом! \n\nВысокая степень готовности дома. Сдача дома 4 квартал 2023 год\nРассрочка от застройщика. Материнский капитал. \n\nПрекрасное вложение как для проживания, так и для сдачи в аренду\nЗвоните! Продам квартиру в Севастополе.",1,2022-09-07 14:11:37.645
1,"2-комн. кв., 55,2 м², 4/10 этаж",Секция 5 • Сдан,дом сдан,10 692 000 ₽,193 696 ₽/м²,"[Севастополь, р-н Гагаринский, мкр. Камышовая бухта, улица Павла Корчагина, 23]","[4 часа назад, сегодня, 09:36]",https://sevastopol.cian.ru/sale/flat/273539447/,"Продаётся 2-комнатная квартира в сданном доме (Дом 1), срок сдачи: II-кв. 2022, общей площадью 55.2 кв.м., на 4 этаже. Новый жилой комплекс ""Апельсин"" расположен на улице Корчагина в 10 минутах от моря и самого оборудованного пляжа Севастополя. Дизайн дома задуман в едином стиле с соседними объектами, но обладает своими уникальными особенностями. \nАРХИТЕКТУРНЫЙ СТИЛЬ \nРасцветка дома выполнена в светлых южных оттенках в тон соседним ЖК ""Каравелла"" и ЖК ""Дом на Корчагина"". Благодаря этому жилые комплексы превратятся в полноценный микрорайон, возведенный в единой концепции. \nБЕЗУПРЕЧНАЯ ИНФРАСТРУКТУРА \nВ районе жилого комплекса располагаются удобные транспортные развязки, остановки общественного транспорта, магазины, банки, аптеки. Рядом находятся социальные объекты - школа и детский сад. \nМЕСТА ДЛЯ РАЗВЛЕЧЕНИЙ \nВозле жилого комплекса расположен любимый у детей ""Динопарк"" с площадками для активного отдыха. Взрослые смогут посещать спектакли и концерты в Культурно-информационном центре, куда регулярно приезжают с гастролями российские звезды. В пяти минутах ходьбы от объекта находится крупный торговый центр ""Апельсин"" со множеством магазинов и одним из самых популярных кинотеатров города. \nБЛИЗОСТЬ МОРЯ \nНеподалеку от ЖК ""Апельсин"" расположен популярный галечный пляж Севастополя - ""Адмиральская лагуна"". Дорога к нему проходит мимо новых уютных парков и современных жилых домов и занимает всего несколько минут. \nУлица Корчагина удачно сочетает в себе динамичный ритм молодого микрорайона и спокойную гармонию прибрежной зоны. А свежий черноморский воздух обеспечит прекрасное настроение в любое время года. Продается квартира в Севастополе.",1,2022-09-07 14:11:37.645
4,"1-комн. апарт., 42,65 м², 1/3 этаж",Сдача корпуса 4 кв. 2022,сдача ГК: 4 кв. 2022 года,2 559 000 ₽,60 000 ₽/м²,"[Севастополь, р-н Гагаринский, Казачья Бухта мкр]","[4 часа назад, сегодня, 10:04]",https://sevastopol.cian.ru/sale/flat/273181071/,"Дом ""Авангард"" отличается уникальной архитектурой и планировками.\nВ доме 2 трехэтажные секции. Дом построен по монолитно-каркасной технологии.\nЖК ""Скифия"" расположен в одном из самых перспективных и динамично развивающихся районов города в курортной части бухты Казачья в Гагаринском районе Севастополя. Благодаря сочетанию качества, экологичности и доступно

---

In [ ]:
# from lib.downloader import DownloaderSimple
# html = DownloaderSimple().get(url)
# with open('tmp/cian_.html','wt') as f: f.write(html[0])

---

In [ ]:
# <article data-name="CardComponent" 

In [ ]:
# <span data-mark="OfferTitle"
# <span data-mark="OfferSubtitle" 
# <span data-mark="Deadline"

# <a data-name="GeoLabel"

# <span data-mark="MainPrice"

# <p data-mark="PriceInfo" 

# <div data-name="Description" 

# <div data-name="LinkArea" class="_93444fe79c--container--kZeLu _93444fe79c--link--DqDOy">
# <a href="https://sevastopol.cian.ru/sale/flat/273181085/" class="_93444fe79c--link--eoxce">

# <div data-name="TimeLabel" 

In [ ]:
# def get_link(tag):
#     return tag.find('a').attrs['href']

# def get_ts(tag):
#     return [ t.text for t in tag.find_all('span') ]

# def get_adr(tag):
#     return [ t.text for t in tag.find_all('a',{'data-name':'GeoLabel',}) ]

# def parse_item(tag):
#     return {
#     'OfferTitle':tag.find('span',{'data-mark':'OfferTitle',}).text,
    
#     #'OfferSubtitle': tag.find('span',{'data-mark':'OfferSubtitle',}).text,
#     # 'Deadline': tag.find('span',{'data-mark':'Deadline',}).text,
    
#     'GeoLabel':  get_adr(tag),  # tag.find('span',{'data-name':'GeoLabel',}).text,
        
#     'MainPrice': tag.find('span',{'data-mark':'MainPrice',}).text,
#     'PriceInfo': tag.find('p',{'data-mark':'PriceInfo',}).text,
#     'Description': tag.find('div',{'data-name':'Description',}).text,
#     'LinkArea': get_link( tag.find('div',{'data-name':'LinkArea',})), 
#     'TimeLabel': get_ts( tag.find('div',{'data-name':'TimeLabel',})),  
#     }
    
    

In [ ]:
# root = BeautifulSoup(html,'html.parser')
# text = [
#     parse_item(tag)
#     for tag in root.find_all('article',{'data-name':'CardComponent',}) 
# ]


In [ ]:
# pd.DataFrame(text)

In [ ]:
# text[0]

In [ ]:
# text[1]

In [ ]:
# from lib.downloader import DownloaderSimple

# df,html = CianParser(driver=DownloaderSimple()).load(req_param, keep_html=True, page_limit=3)

# # html = DownloaderSimple().get(url)
# with open('tmp/cian.html','wt') as f: f.write(html)